In [8]:
%%writefile backend.py
# ============================================================
# 🧠 backend.py — Twitter Sentiment Analysis helpers (fixed)
# ============================================================
# This file defines functions only (no top-level file upload or heavy work).
# Call the functions from your Streamlit UI (app.py).
# If executed directly (python backend.py) it will run the full pipeline.

import os
import re
import time
import sqlite3
import multiprocessing
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
from email.mime.base import MIMEBase
from email import encoders
import nltk
import spacy
import torch
from transformers import pipeline

# -- Ensure minimal NLTK downloads (quiet)
nltk.download("vader_lexicon", quiet=True)
nltk.download("stopwords", quiet=True)
nltk.download("wordnet", quiet=True)

# ------------------------------------------------------------
# Configuration defaults
# ------------------------------------------------------------
DEFAULT_CSV = "tweetsData.csv"
DEFAULT_DB = "tweets.db"
SENT_PIE_PATH = "sentiment_pie.png"
OUTPUT_CSV = "Milestone3_Final_Results.csv"

# ------------------------------------------------------------
# Utility: load CSV
# ------------------------------------------------------------
def load_csv(csv_path=DEFAULT_CSV):
    """Load CSV from path. Raises FileNotFoundError if not present."""
    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"{csv_path} not found.")
    df = pd.read_csv(csv_path)
    return df

# ------------------------------------------------------------
# Utility: save DataFrame to SQLite table
# ------------------------------------------------------------
def save_df_to_db(df, db_path=DEFAULT_DB, table="tweets_table"):
    conn = sqlite3.connect(db_path)
    try:
        df.to_sql(table, conn, if_exists="replace", index=False)
    finally:
        conn.close()

# ------------------------------------------------------------
# Utility: load table from DB
# ------------------------------------------------------------
def load_table_from_db(db_path=DEFAULT_DB, table="tweets_table"):
    if not os.path.exists(db_path):
        raise FileNotFoundError(f"{db_path} not found.")
    conn = sqlite3.connect(db_path)
    try:
        df = pd.read_sql_query(f"SELECT * FROM {table}", conn)
    finally:
        conn.close()
    return df

# ------------------------------------------------------------
# 1️⃣ Cleaning function (same logic)
# ------------------------------------------------------------
def clean_text(text):
    """Lowercase, remove links, mentions, non-alpha characters."""
    text = str(text).lower()
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"[^a-z\s]", "", text)
    return text.strip()

# ------------------------------------------------------------
# 2️⃣ Normalization (spaCy lemmatization)
# Lazy-load spaCy model on first call to avoid import-time cost.
# ------------------------------------------------------------
_spacy_nlp = None

def _get_spacy():
    global _spacy_nlp
    if _spacy_nlp is None:
        _spacy_nlp = spacy.load("en_core_web_sm")
    return _spacy_nlp

def token_lemma_nonstop(text):
    """Tokenize, lemmatize, remove stopwords using spaCy."""
    nlp = _get_spacy()
    doc = nlp(str(text))
    tokens = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]
    return " ".join(tokens)

# ------------------------------------------------------------
# 3️⃣ Word count with optional multiprocessing
# ------------------------------------------------------------
def _count_chunk(chunk):
    c = Counter()
    for t in chunk:
        c.update(str(t).split())
    return c

def count_words_list(texts, num_processes=4):
    """Return Counter of word frequencies (uses multiprocessing)."""
    if not texts:
        return Counter()

    n = max(1, num_processes)
    chunk_size = max(1, len(texts) // n)
    chunks = [texts[i:i+chunk_size] for i in range(0, len(texts), chunk_size)]

    if len(chunks) == 1:
        results = [_count_chunk(chunks[0])]
    else:
        with multiprocessing.Pool(processes=n) as pool:
            results = pool.map(_count_chunk, chunks)

    total = sum(results, Counter())
    return total

# ------------------------------------------------------------
# 4️⃣ VADER Sentiment (parallel-safe for Colab)
# ------------------------------------------------------------
from nltk.sentiment import SentimentIntensityAnalyzer
_sia = SentimentIntensityAnalyzer()

def _vader_analyze_chunk(chunk):
    """Helper for multiprocessing — process one chunk of text."""
    out = []
    for t in chunk:
        score = _sia.polarity_scores(str(t))["compound"]
        if score > 0.05:
            label = "positive"
        elif score < -0.05:
            label = "negative"
        else:
            label = "neutral"
        out.append((score, label))
    return out

def vader_sentiment_texts(texts, num_processes=4):
    """Given list of texts, return list of (score, label) tuples in same order."""
    if not texts:
        return []

    n = max(1, num_processes)
    chunk_size = max(1, len(texts) // n)
    chunks = [texts[i:i + chunk_size] for i in range(0, len(texts), chunk_size)]

    if len(chunks) == 1:
        results = [_vader_analyze_chunk(chunks[0])]
    else:
        with multiprocessing.Pool(processes=n) as pool:
            results = pool.map(_vader_analyze_chunk, chunks)

    # flatten
    flat = [item for sub in results for item in sub]
    return flat

# ------------------------------------------------------------
# 5️⃣ Save sentiment results to DB
# ------------------------------------------------------------
def save_sentiment_results(df, db_path=DEFAULT_DB, table="tweets_table_sentiment"):
    conn = sqlite3.connect(db_path)
    try:
        df.to_sql(table, conn, if_exists="replace", index=False)
    finally:
        conn.close()

# ------------------------------------------------------------
# 6️⃣ Plot helpers: bar and pie (return fig object)
# ------------------------------------------------------------
def plot_top_words(counter, top_n=10):
    top = counter.most_common(top_n)
    if not top:
        fig = plt.figure()
        return fig
    words, counts = zip(*top)
    fig, ax = plt.subplots(figsize=(10,5))
    ax.bar(words, counts, edgecolor="black")
    plt.xticks(rotation=45, ha="right")
    plt.xlabel("Words")
    plt.ylabel("Frequency")
    plt.title(f"Top {top_n} Most Frequent Words")
    plt.tight_layout()
    return fig

def plot_sentiment_pie(labels_series, save_path=SENT_PIE_PATH):
    counts = labels_series.value_counts()
    fig, ax = plt.subplots(figsize=(6,6))
    ax.pie(counts, labels=counts.index, autopct="%1.1f%%", startangle=140,
           colors=["lightgreen","salmon","lightblue"])
    ax.set_title("Sentiment Distribution (VADER)")
    fig.savefig(save_path, bbox_inches="tight")
    plt.close(fig)
    return save_path

# ------------------------------------------------------------
# 7️⃣ Email helpers (image or csv)
# ------------------------------------------------------------
def send_email_alert(to_email, subject, message, image_path=None,
                     smtp_server="smtp.gmail.com", smtp_port=587,
                     sender_email=None, sender_password=None):
    """
    Send an email with optional image attachment.
    Note: supply sender_email and sender_password (app password) when calling.
    """
    if sender_email is None or sender_password is None:
        raise ValueError("sender_email and sender_password must be provided")

    msg = MIMEMultipart()
    msg["From"] = sender_email
    msg["To"] = to_email
    msg["Subject"] = subject
    msg.attach(MIMEText(message, "plain"))

    if image_path and os.path.exists(image_path):
        with open(image_path, "rb") as f:
            img = MIMEImage(f.read())
            img.add_header('Content-Disposition', 'attachment', filename=os.path.basename(image_path))
            msg.attach(img)

    server = smtplib.SMTP(smtp_server, smtp_port)
    try:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, to_email, msg.as_string())
    finally:
        server.quit()

def send_email_with_csv(to_email, subject, message, file_path=None,
                        smtp_server="smtp.gmail.com", smtp_port=587,
                        sender_email=None, sender_password=None):
    """
    Send email with CSV attachment.
    """
    if sender_email is None or sender_password is None:
        raise ValueError("sender_email and sender_password must be provided")

    msg = MIMEMultipart()
    msg["From"] = sender_email
    msg["To"] = to_email
    msg["Subject"] = subject
    msg.attach(MIMEText(message, "plain"))

    if file_path and os.path.exists(file_path):
        with open(file_path, "rb") as f:
            part = MIMEBase("application", "octet-stream")
            part.set_payload(f.read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition', f'attachment; filename="{os.path.basename(file_path)}"')
        msg.attach(part)

    server = smtplib.SMTP(smtp_server, smtp_port)
    try:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, to_email, msg.as_string())
    finally:
        server.quit()

# ------------------------------------------------------------
# 8️⃣ Hugging Face sentiment (modern)
# ------------------------------------------------------------
# ------------------------------------------------------------
# 8️⃣ Hugging Face sentiment (optimized for Colab / Streamlit)
# ------------------------------------------------------------
from transformers import pipeline

def huggingface_sentiment(texts, model_name="cardiffnlp/twitter-roberta-base-sentiment", device=None, batch_size=32):
    """
    Run Hugging Face sentiment analysis efficiently (no multiprocessing).
    Handles batching internally and avoids heavy parallel model loads.
    Returns list of dicts like {'label':..., 'score':...}.
    """
    if not texts:
        return []

    if device is None:
        device = 0 if torch.cuda.is_available() else -1

    # load pipeline once
    pipe = pipeline("sentiment-analysis", model=model_name, device=device)

    results = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        res = pipe(batch, truncation=True)
        label_map = {"LABEL_0": "Negative", "LABEL_1": "Neutral", "LABEL_2": "Positive"}
        mapped = [{"label": label_map.get(r["label"], r["label"]), "score": r["score"]} for r in res]
        results.extend(mapped)

    return results

# ------------------------------------------------------------
# 9️⃣ Agreement check & confusion table
# ------------------------------------------------------------
def agreement_table(df, col_a="sentiment_label", col_b="modern_label"):
    if col_a not in df.columns or col_b not in df.columns:
        raise KeyError(f"Columns {col_a} and/or {col_b} not in DataFrame")
    agreement = (df[col_a] == df[col_b]).mean()
    table = pd.crosstab(df[col_a], df[col_b])
    return agreement, table

# ------------------------------------------------------------
# 10️⃣ Convenience: full pipeline runner (only when executed directly)
# ------------------------------------------------------------
def run_full_pipeline(csv_path=DEFAULT_CSV, db_path=DEFAULT_DB, sender_email=None, sender_password=None):
    """
    Runs the full pipeline similar to your original script.
    Saves intermediate artifacts and returns final dataframe.
    """
    # 1. load
    df = load_csv(csv_path)

    # 2. save to db
    save_df_to_db(df, db_path=db_path, table="tweets_table")

    # 3. cleaning
    df["cleaned"] = df["text"].apply(clean_text)

    # 4. normalization
    df["normalized"] = df["cleaned"].apply(token_lemma_nonstop)

    # 5. word counts
    texts_norm = df["normalized"].astype(str).tolist()
    total_counts = count_words_list(texts_norm, num_processes=4)

    # 6. vader sentiment
    vader_results = vader_sentiment_texts(df["text"].astype(str).tolist(), num_processes=4)
    if vader_results:
        df["sentiment_score"], df["sentiment_label"] = zip(*vader_results)
    else:
        df["sentiment_score"], df["sentiment_label"] = [], []

    # 7. save sentiment db and pie
    save_sentiment_results(df, db_path=db_path, table="tweets_table_sentiment")
    plot_sentiment_pie(df["sentiment_label"], save_path=SENT_PIE_PATH)

    # 8. hugging face sentiment
    hf_results = huggingface_sentiment(df["text"].astype(str).tolist())
    df["modern_label"] = [r["label"] for r in hf_results]
    df["modern_score"] = [r["score"] for r in hf_results]

    # ✅ Rename columns for consistency with UI
    df.rename(columns={
        "sentiment_label": "vader_label",
        "modern_label": "hf_label"
    }, inplace=True)

    # 9. save CSV output
    df.to_csv(OUTPUT_CSV, index=False)

    # 10. optionally email results (if credentials provided)
    summary_lines = [
        "📊 Twitter Sentiment Report\n\n",
        f"Total tweets analyzed: {len(df)}\n",
        f"Sentiment counts: {df['sentiment_label'].value_counts().to_dict()}\n\n",
        f"Top words (normalized): {total_counts.most_common(10)}\n\n"
    ]
    summary_text = "".join(summary_lines)

    if sender_email and sender_password:
        try:
            send_email_alert(sender_email, "Twitter Sentiment Report", summary_text, image_path=SENT_PIE_PATH,
                             sender_email=sender_email, sender_password=sender_password)
        except Exception as e:
            print("Email send failed:", e)

    return df

def build_final_results(db_path=DEFAULT_DB, output_csv=OUTPUT_CSV):
    """
    Merge VADER and Hugging Face results (if available) into final CSV.
    Returns DataFrame of merged results.
    """
    if not os.path.exists(db_path):
        raise FileNotFoundError("Database not found. Please run analysis first.")

    try:
        df_vader = load_table_from_db(db_path, table="tweets_vader")
        df_hf = load_table_from_db(db_path, table="tweets_hf")
        df_final = pd.merge(df_vader, df_hf, on="text", suffixes=("_vader", "_hf"))
        df_final.to_csv(output_csv, index=False)
        return df_final
    except Exception as e:
        raise RuntimeError(f"Failed to build final results: {e}")
# ------------------------------------------------------------
# Run full pipeline only when file executed directly
# ------------------------------------------------------------
if __name__ == "__main__":
    # If user runs `python backend.py`, attempt to run full pipeline with defaults.
    try:
        print("Running full backend pipeline (from backend.py)...")
        run_full_pipeline()
        print("Done.")
    except Exception as e:
        print("Error running full pipeline:", e)


Overwriting backend.py


In [15]:
%%writefile app.py
# ============================================================
# 🌟 Twitter Sentiment Analyzer — Modern Streamlit UI
# ============================================================

import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import os   # ✅ Added for Colab secret access
from io import BytesIO
from backend import (
    clean_text,
    token_lemma_nonstop,
    count_words_list,
    vader_sentiment_texts,
    huggingface_sentiment,
    plot_top_words,
    plot_sentiment_pie,
    send_email_with_csv,
    save_df_to_db,
    load_table_from_db,
    save_sentiment_results,
    agreement_table,
)

# ============================================================
# PAGE CONFIG
# ============================================================
st.set_page_config(page_title="Twitter Sentiment Analyzer", layout="wide")
st.markdown(
    """
    <style>
    .main-title {
        font-size: 2.2rem;
        color: #2E8B57;
        text-align: center;
        font-weight: 700;
        margin-bottom: 0.3rem;
    }
    .sub-title {
        text-align: center;
        color: #444;
        margin-bottom: 1.5rem;
    }
    .footer {
        text-align: center;
        margin-top: 2rem;
        color: gray;
        font-size: 0.9rem;
    }
    </style>
    """,
    unsafe_allow_html=True,
)

st.markdown('<div class="main-title">🧠 Twitter Sentiment Analyzer</div>', unsafe_allow_html=True)
st.markdown('<div class="sub-title">Upload → Clean → Analyze → Compare → Download or Email Results</div>', unsafe_allow_html=True)

DB_PATH = "tweets.db"
RESULT_CSV = "Milestone3_Final_Results.csv"

# ============================================================
# MAIN APP
# ============================================================
tabs = st.tabs(["📁 Upload Data", "🧹 Clean & Normalize", "📊 Top Words", "🤖 Sentiment Analysis", "⚖️ Comparison", "📧 Report"])

# ============================================================
# TAB 1: UPLOAD
# ============================================================
with tabs[0]:
    st.header("📁 Upload Your CSV File")
    uploaded_file = st.file_uploader("Upload tweetsData.csv", type=["csv"])

    if uploaded_file:
        df = pd.read_csv(uploaded_file)
        st.success(f"✅ File uploaded successfully — {len(df)} rows, {len(df.columns)} columns")

        # Save to DB
        save_df_to_db(df, DB_PATH, table="tweets_table")

        # Preview section
        st.subheader("👀 Data Preview")
        num_rows = st.slider("Select number of rows to display", 5, 50, 10)
        st.dataframe(df.head(num_rows), use_container_width=True)

        st.info("Next: Go to the 'Clean & Normalize' tab to preprocess text.")
    else:
        st.warning("Please upload a CSV file to continue.")

# ============================================================
# TAB 2: CLEAN & NORMALIZE (Fixed – cleaned preview persists)
# ============================================================
with tabs[1]:
    st.header("🧹 Clean & Normalize Text ")
    if os.path.exists(DB_PATH):
        df = load_table_from_db(DB_PATH)
        col1, col2 = st.columns(2)

        # Initialize session states if missing
        if "cleaned_df" not in st.session_state:
            st.session_state.cleaned_df = None
        if "normalized_df" not in st.session_state:
            st.session_state.normalized_df = None

        with col1:
            if st.button("🧼 Clean Text"):
                with st.spinner("Cleaning tweets..."):
                    df["cleaned"] = df["text"].apply(clean_text)
                    save_df_to_db(df, DB_PATH, table="tweets_table_cleaned")
                    st.session_state.cleaned_df = df[["text", "cleaned"]]
                st.success("✅ Text cleaned successfully!")

        with col2:
            if st.button("Normalize Text"):
                with st.spinner("Normalizing text..."):
                    # Make sure cleaned data exists
                    if "cleaned" not in df.columns:
                        if st.session_state.cleaned_df is not None:
                            df = st.session_state.cleaned_df.copy()
                        else:
                            st.warning("Cleaned text not found — please clean first.")
                            st.stop()

                    df["normalized"] = df["cleaned"].apply(token_lemma_nonstop)
                    save_df_to_db(df, DB_PATH, table="tweets_table_cleaned")
                    st.session_state.normalized_df = df[["text", "cleaned", "normalized"]]
                st.success("✅ Text normalized successfully!")

        # Always show previews if available
        if st.session_state.cleaned_df is not None:
            st.subheader("🧼 Cleaned Text Preview")
            st.dataframe(st.session_state.cleaned_df.head(10), width="stretch")

        if st.session_state.normalized_df is not None:
            st.subheader("🔤 Normalized Text Preview")
            st.dataframe(st.session_state.normalized_df.head(10), width="stretch")

    else:
        st.warning("⚠️ Please upload data first in the 'Upload Data' tab.")


# ============================================================
# TAB 3: TOP WORDS (FULLY FIXED)
# ============================================================
with tabs[2]:
    st.header("📊 View Top Frequent Words")
    if os.path.exists(DB_PATH):
        # Load cleaned/normalized table explicitly
        df = load_table_from_db(DB_PATH, table="tweets_table_cleaned")

        if "normalized" in df.columns and not df["normalized"].isna().all():
            st.info("✅ Using normalized text for word frequency analysis.")
            top_n = st.number_input("Number of top words to display", min_value=5, max_value=50, value=10)
            if st.button("Show Top Words"):
                with st.spinner("Counting top words..."):
                    texts_norm = df["normalized"].astype(str).tolist()
                    total_counts = count_words_list(texts_norm)
                    fig = plot_top_words(total_counts, top_n=top_n)
                    st.pyplot(fig)
        else:
            st.warning("⚠️ Please normalize the text first in the previous tab.")
    else:
        st.warning("⚠️ Please upload and clean your data first.")

# ============================================================
# TAB 4: SENTIMENT ANALYSIS
# ============================================================
with tabs[3]:
    st.header("🤖 Sentiment Analysis Options")

    if os.path.exists(DB_PATH):
        df = load_table_from_db(DB_PATH)
        sub_tabs = st.tabs(["⚙️ VADER Analysis", "🤖 Hugging Face Analysis"])

        # ---------- VADER ----------
        with sub_tabs[0]:
            if st.button("Run VADER Sentiment Analysis"):
                with st.spinner("Running VADER sentiment..."):
                    results = vader_sentiment_texts(df["text"].astype(str).tolist(), num_processes=4)
                    if results:
                        df["vader_score"], df["vader_label"] = zip(*results)
                        save_sentiment_results(df, DB_PATH, table="tweets_vader")
                        pie_path = plot_sentiment_pie(df["vader_label"])
                        st.image(pie_path, caption="VADER Sentiment Distribution")
                        st.dataframe(df[["text", "vader_label", "vader_score"]].head(10))
                        st.success("✅ VADER Sentiment Analysis Completed!")
                        df.to_csv("vader_results.csv", index=False)
                        st.download_button("📥 Download VADER Results", data=open("vader_results.csv", "rb"), file_name="vader_results.csv")
                    else:
                        st.error("No sentiment results found.")

        # ---------- HUGGING FACE ----------
        with sub_tabs[1]:
            if st.button("Run Hugging Face Sentiment Analysis"):
                with st.spinner("Running Hugging Face model... this may take a minute ⏳"):
                    results = huggingface_sentiment(df["text"].astype(str).tolist())
                    df["hf_label"] = [r["label"] for r in results]
                    df["hf_score"] = [r["score"] for r in results]
                    save_sentiment_results(df, DB_PATH, table="tweets_hf")
                    pie_path = plot_sentiment_pie(df["hf_label"])
                    st.image(pie_path, caption="Hugging Face Sentiment Distribution")
                    st.dataframe(df[["text", "hf_label", "hf_score"]].head(10))
                    st.success("✅ Hugging Face Sentiment Completed!")
                    df.to_csv("hf_results.csv", index=False)
                    st.download_button("📥 Download HF Results", data=open("hf_results.csv", "rb"), file_name="hf_results.csv")
    else:
        st.warning("Please upload and clean your data first.")

# ============================================================
# TAB 5: COMPARISON (FIXED)
# ============================================================
with tabs[4]:
    st.header("⚖️ Compare VADER vs Hugging Face")

    if os.path.exists(DB_PATH):
        try:
            # Load both sentiment tables
            df_vader = load_table_from_db(DB_PATH, table="tweets_vader")
            df_hf = load_table_from_db(DB_PATH, table="tweets_hf")

            # Merge on 'text' column (assuming tweets are same)
            df_merged = pd.merge(df_vader, df_hf, on="text", suffixes=("_vader", "_hf"))

            if "vader_label" in df_merged.columns and "hf_label" in df_merged.columns:
                agreement, table = agreement_table(df_merged, col_a="vader_label", col_b="hf_label")
                st.metric(label="Model Agreement (%)", value=f"{agreement*100:.2f}")

                fig, ax = plt.subplots(figsize=(6, 4))
                sns.heatmap(table, annot=True, fmt="d", cmap="Greens", ax=ax)
                plt.title("VADER vs HF Sentiment Comparison")
                st.pyplot(fig)

                st.download_button(
                    "📥 Download Comparison Table",
                    data=table.to_csv().encode('utf-8'),
                    file_name="comparison_table.csv"
                )
            else:
                st.warning("Please run both sentiment analyses first.")
        except Exception as e:
            st.error(f"Error loading comparison: {e}")
    else:
        st.warning("Please complete previous steps first.")


# ============================================================
# TAB 6: REPORT / EMAIL (Updated Secure Email)
# ============================================================
with tabs[5]:
    st.header("📧 Download or Email Your Results")

    if os.path.exists(DB_PATH):
        try:
            from backend import build_final_results

            # Use backend function to generate final CSV
            df_final = build_final_results(DB_PATH, RESULT_CSV)
            st.success(f"✅ Final report ready with {len(df_final)} tweets!")

            # -------------------------------
            # Download Section
            # -------------------------------
            st.markdown("### 📥 Download Final CSV")
            with open(RESULT_CSV, "rb") as f:
                st.download_button("Download File", f, file_name=RESULT_CSV)

            # -------------------------------
            # Email Section (Secure)
            # -------------------------------
            st.markdown("### 📩 Email Report")
            email = st.text_input("Enter your email address:")
            attach_chart = st.checkbox("Attach Sentiment Pie Chart")

            if st.button("Send Email Report"):
                if not email:
                    st.error("Please enter a valid email address.")
                else:
                    msg = f"Twitter Sentiment Report — {len(df_final)} Tweets analyzed successfully!"
                    image_path = "sentiment_pie.png" if attach_chart and os.path.exists("sentiment_pie.png") else None

                    # ✅ Secure method: Load Gmail password from Colab Secret
                    SENDER_EMAIL = "muskansoni0524@gmail.com"
                    SENDER_PASSWORD = os.environ.get("gmail_password")

                    if not SENDER_PASSWORD:
                        st.error("⚠️ Gmail password secret not found. Please add it as a Colab secret named 'gmail_password'.")
                    else:
                        try:
                            send_email_with_csv(
                                to_email=email,
                                subject="Twitter Sentiment Report",
                                message=msg,
                                file_path=RESULT_CSV,
                                sender_email=SENDER_EMAIL,
                                sender_password=SENDER_PASSWORD
                            )
                            st.success(f"✅ Report emailed successfully to {email}")
                        except Exception as e:
                            st.error(f"❌ Failed to send email: {e}")

        except Exception as e:
            st.warning(f"⚠️ Could not prepare final report: {e}")
            st.info("Please ensure both VADER and Hugging Face analyses were run.")
    else:
        st.warning("⚠️ Please upload and analyze your data before generating a report.")


Overwriting app.py


In [16]:
!pip install streamlit pyngrok --quiet

In [ ]:
from pyngrok import ngrok

# 🔑 STEP 1 — Set your ngrok auth token (only once per runtime)
ngrok.set_auth_token("34T9eLS4rcH76eKVZwXEKFv4uLa_456R8DWTQcoGd5govmc7o")  # 👈 paste your token here

# 🛑 STEP 2 — Kill any existing tunnels (safety)
ngrok.kill()

# 🌍 STEP 3 — Create a new tunnel to Streamlit's default port 8501
public_url = ngrok.connect(8501)
print(f"🌍 Your app is live at: {public_url}")

# 🚀 STEP 4 — Run your Streamlit app
!streamlit run app.py --server.port 8501


🌍 Your app is live at: NgrokTunnel: "https://unstated-rudolf-unbrazenly.ngrok-free.dev" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.11.229.54:8501

2025-11-05 15:19:08.945822: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762355948.967281   26428 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762355948.973761   26428 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762355948.991099   26428 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the 